In [26]:
import logging

from torch.nn.modules.pooling import MaxPool1d
logging.basicConfig(level=logging.INFO)

import heapq
from pathlib import Path
import gzip

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import sentencepiece as spm

from tp8_preprocess import TextDataset

# Utiliser tp8_preprocess pour générer le vocabulaire BPE et
# le jeu de donnée dans un format compact

# --- Configuration

# Taille du vocabulaire
vocab_size = 1000
try:
    MAINDIR = Path(__file__).parent
except NameError:
    MAINDIR = Path("/tempory/M2-DAC/AMAL/TME/TME8/src/tp8.py").parent
# Chargement du tokenizer

tokenizer = spm.SentencePieceProcessor()
tokenizer.Load(f"wp{vocab_size}.model")
ntokens = len(tokenizer)

def loaddata(mode):
    with gzip.open(f"{mode}-{vocab_size}.pth", "rb") as fp:
        return torch.load(fp)


test = loaddata("test")
train = loaddata("train")
TRAIN_BATCHSIZE=500
TEST_BATCHSIZE=500


# --- Chargements des jeux de données train, validation et test

val_size = 1000
train_size = len(train) - val_size
train, val = torch.utils.data.random_split(train, [train_size, val_size])

logging.info("Datasets: train=%d, val=%d, test=%d", train_size, val_size, len(test))
logging.info("Vocabulary size: %d", vocab_size)
train_iter = torch.utils.data.DataLoader(train, batch_size=TRAIN_BATCHSIZE, collate_fn=TextDataset.collate)
val_iter = torch.utils.data.DataLoader(val, batch_size=TEST_BATCHSIZE, collate_fn=TextDataset.collate)
test_iter = torch.utils.data.DataLoader(test, batch_size=TEST_BATCHSIZE, collate_fn=TextDataset.collate)

INFO:root:Datasets: train=1599201, val=799, test=359
INFO:root:Vocabulary size: 1000


In [27]:
x, y = next(next(iter(train_iter)))
x.size(), y.size(), TRAIN_BATCHSIZE

(torch.Size([500, 59]), torch.Size([500]), 500)

In [28]:
train_raw_data = loaddata("train")

In [29]:
import pandas as pd
pd.Series(train_raw_data.labels).value_counts()

0    800000
1    800000
dtype: int64

In [30]:
pd.Series(loaddata('test').labels).value_counts()

1    182
0    177
dtype: int64

In [ ]:
class textCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.seq = nn.Sequential([
            nn.Conv1d(1, 64),
            nn.MaxPool1d(),
        ])
        self.conv1 = 
        self.pool1 = 
    
    def forward(self, x):
        pass

In [32]:
nn.Embedding(vocab_size, 64)(x).size()

torch.Size([500, 59, 64])

In [33]:
x.size()

torch.Size([500, 59])

In [37]:
for i, (x, y) in enumerate(train_iter):
    print(nn.Embedding(vocab_size, 64)(x).size())
    if i >= 2:
        break


torch.Size([500, 59, 64])
torch.Size([500, 63, 64])
torch.Size([500, 52, 64])
